I would like to investigate the general question 3, about which time windows are likely to be the most busy, posed by the research group led by Frank Wood. The specifc question I will answer based on their general question is: What is the start hour with the most people?

The sessions file will provide me with information about player's gaming session start times, end times, and the duration of their sessions once I calculate them.
I should be able tp predict the time of day with the most traffic based on the count for each hour of players' start times.

I will wrangle the data from sessions.csv so that I have separate columns for date and time, because I am interested in the time and not necessarily the date of play. I will also create a variable about session duration by subtracting session start time from end time for each observation.

In [14]:
library(tidyverse)
library(repr)

In [15]:
players_data <- read_csv("players.csv")
head(players_data)
sessions_data <- read_csv("sessions.csv")
head(sessions_data)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


Players Data Set
7 columns/variables
196 rows/observations
4 character vectors: experience, name, hashedEmail, gender 
2 double vectors for decimal numbers: Age, played_hours 
1 logical vector: subscribe i.e. whether the player is subscribed to the newletter or not
The experience variable is a categorical variable with labels: pro, veteran, regular, amateur, or beginner. Gender is also a categorical variable with 7 options: male, female, agender, non-binary, two-spirited, other, and prefer not to say. 
Issues with this data set are that experience is not a factor vector like it should be since it is a categorical variable, the same could also be said for gender. Another issue is that if you wanted to use experience as a predictor you would have to convert its labels to numbers so calculations could be performed.

Sessions Data Set
5 columns/variables
1535 rows/observations
3 chracter vectors: hashedEmail, start_time, end_time
2 double vectors: orginial_start_time, original_end_time
The start_time and end_time vectors include both day/month/year dates as well as a 24-hour time. It is unclear exactly what units are being use for original_start/end_time but they are not comprehensible to a human.

In [17]:
#add code to calculate averages for number columns here 
players_avgs <- players_data |>
    select(played_hours, Age)|>
    map_dfr( mean, na.rm =TRUE)
   
players_max <- players_data|>
    select(played_hours, Age)|>
    map_dfr(max, na.rm = TRUE)
   
players_min <- players_data|>
    select(played_hours, Age)|>
    map_dfr(min, na.rm = TRUE)

players_avgs
players_min
players_max
#separate dates from times, separates hours from minutes, convert minutes to hours, combine hours and minutes, take average. ?separate dates by days and months 
#and convert units to months and add months to days to see the average date someone started or ended? year doesn't really matter since it all happened in 2024
#maybe just take the average of days months and years separately because the time of month someone joined doesn't really matter just the month itself.
#not wrangling the end_time column because I won't be using it
#make it a space separated files in text editor using replace  use read delim
 sessions_wrang <- sessions_data |>
    strsplit( as.character(sessions_data$start_time), '/s ')|>
    strsplit( as.character(sessions_data$end_time), '/s ')#|>
    # rename()|>#rename new columns to start_date,start_time,end_date,end_time
    # # separate(col = start_time,
    # #          c( "start_date", "start_time"),
    # #          sep = " ", 
    # #          remove = FALSE,
    # #          convert = TRUE)|>
    # separate(col= start_date,
    #          c( "start_day", "start_month", "start_yr"),
    #          sep = "/",
    #          remove = FALSE,
    #          convert = TRUE)|>
    # separate(col= end_date,
    #          c( "end_day", "end_month", "end_yr"),
    #          sep = "/",
    #          remove = FALSE,
    #          convert = TRUE)|>
    # separate(col = start_time,
    #          c( "start_hour", "start_minutes"),
    #         sep = ":", 
    #          remove = FALSE,
    #         convert = TRUE)|>
    # separate(col = end_time,
    #          c( "end_hour", "end_minutes"),
    #         sep = ":", 
    #          remove = FALSE,
    #         convert = TRUE)|>
    # mutate( decimal_start_minutes = start_minutes/60 )|>
    # mutate(decimal_start_hr = decimal_start_minutes + start_hour)|>
    # mutate( decimal_end_minutes = end_minutes/60 )|>
    # mutate(decimal_end_hr = decimal_end_minutes + end_hour)|>
    # mutate(duration_hrs = decimal_end_hr - decimal_start_hr)

sessions_wrang

# sessions_avgs<- sessions_wrang|>
#     select(start_yr, start_month, start_day, decimal_start_hr, original_start_time, original_end_time, end_yr, end_month, end_day, decimal_end_hr, duration_hrs)
#     map_dfr(mean, na.rm = TRUE)

# sessions_avgs


played_hours,Age
<dbl>,<dbl>
5.845918,21.13918


played_hours,Age
<dbl>,<dbl>
0,9


played_hours,Age
<dbl>,<dbl>
223.1,58


ERROR: Error in strsplit(sessions_data, as.character(sessions_data$start_time), : non-character argument


The average age of players is ~21 years, the maximum is 58 years, and the minimum is 9 years old. The average amount of hours played are ~5.8, the maximum is 223.1 hours, and the minimum is 0 hours. 
The average start time of day is . The average start month is . The average start day during the month is . The average start year is 2024. The average end time of day is . The average end month is . The average end day during the month is . The average end year is 2024. The average original_start_time is . The average original_end_time is .

In [8]:
#exploratory visualizations
options(repr.plot.height = 10, repr.plot.width = 10)

start_time_hist <- sessions_wrang|>
    ggplot( aes(x= start_hour))+
    geom_histogram()+
    labs(x= "Session Start Hour", y = "Count", title = "Number of Session Starts by Hour")+
    theme( text = element_text(size = 16))

end_time_hist <- sessions_wrang|>
    ggplot( aes(x= end_hour))+
    geom_histogram()+
    labs(x= "Session End Hour", y = "Count", title = "Number of Session Ends by Hour")+
    theme( text = element_text(size = 16))

duration_hist <- sessions_wrang|>
    ggplot( aes(x= duration_hrs))+
    geom_histogram()+
    labs(x= "Session Duration (hours)", y = "Count", title = "Number of Sessions per Hours of Duration")+
    theme( text = element_text(size = 16))

start_time_hist
end_time_hist
duration_hist

ERROR: Error in experience_vs_start_time: The pipe operator requires a function call as RHS (<text>:6:1)


I plan to predict the hours of highest traffic by observing the hour with the most start sessions and combining this with an observation of the mode of session durations, I can predict the hours of highest traffic during each day. The hour with the most start session will tell me when the high traffic time starts, and the mode of session durations will tell me how long the period of high traffic is likley to last.